In [2]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json
import os.path
from datetime import date
from datetime import datetime

import logging

import requests

import zipfile,fnmatch
import os,os.path,sys

import pickle

In [3]:
def scrape(inicio=0,fim=10):
    
    logger = logging.getLogger('ftpuploader')
    hdlr = logging.FileHandler('ftplog.log')
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr)
    logger.setLevel(logging.INFO)
    FTPADDR = "some ftp address"
    
    
    path='C:/Users/Felipe/Documents/GitHub/IntroDS-Projeto-Final/Data2/'
   
    ## Setting selenium to download pdfs
    options_ = Options()
    options_.set_preference("browser.download.folderList",2)
    options_.set_preference("browser.download.manager.showWhenStarting", False)
    options_.set_preference("browser.download.dir",path)
    options_.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
    options_.set_preference("pdfjs.disabled", "true")
    browser = webdriver.Firefox(options=options_)
    ## End setting
    
    mes_dict={'janeiro':1,'fevereiro':2,'março':3,'abril':4,'maio':5,'junho':6,'julho':7,'agosto':8,'setembro':9,'outubro':10,'novembro':11,'dezembro':12}
    with open('C:/Users/Felipe/Documents/GitHub/IntroDS-Projeto-Final/Data2/Dias_com_sessao_Falta_baixar.txt','rb') as file:
        lista_das_sessoes=pickle.load(file)
        
    print(len(lista_das_sessoes))
    
    for _ in range(inicio,fim):
        
        dia,mes,ano=lista_das_sessoes[0]
        dia=int(dia)
        mes=mes_dict[mes]
        ano=int(ano)
        
        lista_das_sessoes.pop(0)
        
        url1='https://www.camara.leg.br/internet/plenario/notas/notas.asp?dia={0}&mes={1}&ano={2}'.format(dia,mes,ano)
        browser.get(url1)
               
        try:
            h=True
            t=1
            
            while h:        
                try:
                    pdf_get='/html/body/table/tbody/tr[2]/td/ul/strong/li[{0}]/a'.format(t)
                    url_pdf=browser.find_element_by_xpath(pdf_get).get_attribute('href')
                    pdf_name=browser.find_element_by_xpath(pdf_get).text
                    pdf_name=pdf_name.replace(' ','_')
                    pdf_name=pdf_name.replace(':','')
                    pdf_name=pdf_name.replace('-','')
                    arquivo=requests.get(url_pdf)
                   
                    
                    file_name=str(ano)+'_'+str(mes)+'_'+str(dia)+'_'+pdf_name
                    file_=os.path.join(path,file_name+'.pdf')
                    
                    with open(file_,'wb') as file:
                        file.write(arquivo.content)
                    
                    t+=1
                except Exception as e:
                    
                    logger.error('Failed to upload: '+str(e))
                    h=False                   
        except:
            pass
    
    
    print(len(lista_das_sessoes))
    file_name='Dias_com_sessao_Falta_baixar'
    file=os.path.join(path,file_name+'.txt')
            
    with open(file,'wb') as file:
        pickle.dump(lista_das_sessoes,file)

In [4]:
start=time.time()
scrape(0,1500)
end=time.time()
print(end-start)

1500
0
4083.7192845344543
